# 【A/B测试】支付宝营销策略效果分析

## 1. 数据来源
数据集来自阿里云天池：
[阿里云天池 - Audience Expansion Dataset](https://tianchi.aliyun.com/dataset/dataDetail?dataId=50893&lang=zh-cn)

该数据集包含三张表，分别记录了支付宝两组营销策略的活动情况：

+ emb_tb_2.csv: 用户特征数据集
+ effect_tb.csv: 广告点击情况数据集
+ seed_cand_tb.csv: 用户类型数据集
本分析报告主要使用广告点击情况数据，涉及字段如下：

+ dmp_id：营销策略编号（源数据文档未作说明，这里根据数据情况设定为1：对照组，2：营销策略一，3：营销策略二）
+ user_id：支付宝用户ID
+ label：用户当天是否点击活动广告（0：未点击，1：点击）

In [14]:
import pandas as pd
import numpy as np

## 2. 数据处理

### 2.1 数据导入和清洗

In [15]:
# 加载数据，自定义原始数据 header
data = pd.read_csv('https://ds-1300369208.cos.ap-shanghai.myqcloud.com/effect_tb.csv',header = None)
data.columns = ["dt","user_id","label","dmp_id"]

# 日志天数属性用不上，删除该列
data = data.drop(columns = "dt")
data.head(3)

,user_id,label,dmp_id
0,1,0,1
1,1000004,0,1
2,1000004,0,2


In [16]:
# 描述性统计
data.describe()

,user_id,label,dmp_id
count,2.645958e+06,2.645958e+06,2.645958e+06
mean,3.112995e+06,1.456297e-02,1.395761e+00
std,1.828262e+06,1.197952e-01,6.920480e-01
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.526772e+06,0.000000e+00,1.000000e+00
50%,3.062184e+06,0.000000e+00,1.000000e+00
75%,4.721132e+06,0.000000e+00,2.000000e+00
max,6.265402e+06,1.000000e+00,3.000000e+00


### 重复值处理

In [17]:
data.shape

(2645958, 3)

数据行数与独立用户数不统一，检查是否存在重复行：

In [18]:
data[data.duplicated(keep = False)].sort_values(by = ["user_id"])

,user_id,label,dmp_id
8529,1027,0,1
1485546,1027,0,1
1579415,1471,0,1
127827,1471,0,1
404862,2468,0,1
...,...,...,...
1382121,6264633,0,1
1382245,6264940,0,1
2575140,6264940,0,1
1382306,6265082,0,3


In [19]:
data.nunique()

user_id    2410683
label            2
dmp_id           3
dtype: int64

删除重复行，并再次检查

In [20]:
# drop duplicate
data = data.drop_duplicates()

# check if any duplicates left
data[data.duplicated(keep = False)]

,user_id,label,dmp_id


### 空值处理

In [21]:
# check null values
data.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2632975 entries, 0 to 2645957
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   user_id  2632975 non-null  int64
 1   label    2632975 non-null  int64
 2   dmp_id   2632975 non-null  int64
dtypes: int64(3)
memory usage: 80.4 MB


数据集无空值，无需进行处理。

空值的处理流程：[pandas dataframe空值的处理方法](https://zhuanlan.zhihu.com/p/35321806)

### 异常值处理

通过透视表检查各属性字段是否存在不合理取值，同时查看每个分析组别的样本数量

In [22]:
data.pivot_table(index = "dmp_id", columns = "label", values = "user_id",
                aggfunc = "count", margins = True)

label,0,1,All
dmp_id,,,
1,1881745,23918,1905663
2,404811,6296,411107
3,307923,8282,316205
All,2594479,38496,2632975


属性字段未发现异常取值，无需进行处理。

### 2.2 样本容量检验

在进行A/B测试前，需检查样本容量是否满足试验所需最小值。这里借助Evan Miller的样本量计算工具：[Sample Size Calculator](https://www.evanmiller.org/ab-testing/sample-size.html)

首先需要设定点击率基准线以及最小提升比例，我们将对照组的点击率设为基准线.

In [23]:
# 计算对照组的点击率
data[data["dmp_id"] == 1]["label"].mean()

0.012551012429794775

对照组点击率为1.25%，假定我们希望新的营销策略能让广告点击率至少提升1个百分点，则算得所需最小样本量为：2152。

[计算流程](https://www.evanmiller.org/ab-testing/sample-size.html#!1.25;80;5;1;0)

[A/B实验样本量计算原理](https://blog.csdn.net/weixin_41744624/article/details/109840263)

In [24]:
data["dmp_id"].value_counts()

1    1905663
2     411107
3     316205
Name: dmp_id, dtype: int64

两组营销活动的样本量分别为41.11万和31.62万，满足最小样本量需求。

In [25]:
# 保存清洗好的数据备用
data.to_csv("data/output.csv", index = False)